## Sentiment Analysis

In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import asarray
from numpy import zeros
import nltk
import re
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Embedding, LSTM
from keras.layers.core import Dense, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# load data
df = pd.read_csv("./data/training.1600000.processed.noemoticon.csv", encoding = 'latin', header = None)
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
# change columns names
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
# drop useless columns
df = df.drop(['id', 'date', 'flag', 'user'], axis = 1)
df.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [8]:
# count dataset samples to know how many of each class we have
df.target.value_counts()

4    800000
0    800000
Name: target, dtype: int64

In [9]:
def clean_text(text):
    text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    
    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if w not in stop_words]
    
    return " ".join(tokens)
    
df.text = df.text.apply(lambda x: clean_text(x))

In [10]:
df.head()

,target,text
0,0,awww bummer shoulda got david carr third day
1,0,upset update facebook texting might cry result...
2,0,dived many times ball managed save 50 rest go ...
3,0,whole body feels itchy like fire
4,0,behaving mad see


In [11]:
train_size = 0.8

train_data, test_data = train_test_split(df, test_size = 1 - train_size, random_state = 0, stratify = df.target)

In [12]:
print("Train data size: ", len(train_data))
print("Test data size: ", len(test_data))

Train data size:  1280000
Test data size:  320000


In [20]:
print("Train data distr: \n", train_data.target.value_counts())
print("Test data distr: \n", test_data.target.value_counts())

Train data distr: 
 4    640000
0    640000
Name: target, dtype: int64
Test data distr: 
 4    160000
0    160000
Name: target, dtype: int64


In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data.text)

In [22]:
max_length = max([len(s.split()) for s in train_data.text])

x_train = pad_sequences(tokenizer.texts_to_sequences(train_data.text),
                        maxlen = max_length)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data.text),
                       maxlen = max_length)

In [23]:
print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

x_train shape:  (1280000, 50)
x_test shape:  (320000, 50)


In [24]:
# label encoding

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(train_data.target.to_list())

y_train = encoder.transform(train_data.target.to_list())
y_test = encoder.transform(test_data.target.to_list())

y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

y_train shape: (1280000, 1)
y_test shape: (320000, 1)


In [47]:
# embedding layer

# load embedding as a dict
def load_embedding(filename):
    # load embedding into memory, skip first line
    file = open(filename,'r',encoding="utf-8")
    lines = file.readlines()
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = asarray(parts[1:], dtype='float32')
    return embedding

# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = zeros((vocab_size, embedding_dim))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        vector = embedding.get(word)
        if vector is not None:
            weight_matrix[i] = vector
    return weight_matrix

# contains the index for each word
vocab = tokenizer.word_index
# total number of words in our vocabulary, plus one for unknown words
vocab_size = len(tokenizer.word_index) + 1
# embedding dimensions
embedding_dim = 300

# load embedding from file
raw_embedding = load_embedding('./data/glove/glove.6B.300d.txt')
# get vectors in the right order
embedding_matrix = get_weight_matrix(raw_embedding, vocab)

# create the embedding layer
embedding_layer = Embedding(vocab_size, 
                            embedding_dim, 
                            weights = [embedding_matrix], 
                            input_length = max_length, 
                            trainable = False)

In [48]:
# define model

model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(64, 5, activation = "relu"))
model.add(MaxPooling1D(4))
model.add(LSTM(64, return_sequences = True))
model.add(Flatten())
model.add(Dense(500, activation = 'relu'))
model.add(Dense(300, activation = 'relu'))
model.add(Dense(1, activation = "sigmoid"))
print(model.summary())

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 300)           87214200  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 46, 64)            96064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 11, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 11, 64)            33024     
_________________________________________________________________
flatten_1 (Flatten)          (None, 704)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               352500    
_________________________________________________________________
dense_2 (Dense)              (None, 300)              

In [49]:
# train model

batch_s = 1024
num_epochs = 10

ReduceLROnPlateau = ReduceLROnPlateau(factor = 0.1,
                                     min_lr = 0.01,
                                     monitor = 'val_loss',
                                     verbose = 1)

history = model.fit(x_train, y_train, batch_size = batch_s, epochs = num_epochs,
                    validation_data = (x_test, y_test), callbacks = [ReduceLROnPlateau])

Train on 1280000 samples, validate on 320000 samples
Epoch 1/10
1280000/1280000 [==============================] - 82s 64us/step - loss: 0.5821 - accuracy: 0.6822 - val_loss: 0.5726 - val_accuracy: 0.6902
Epoch 2/10
1280000/1280000 [==============================] - 59s 46us/step - loss: 0.5607 - accuracy: 0.7004 - val_loss: 0.5680 - val_accuracy: 0.6938
Epoch 3/10
1280000/1280000 [==============================] - 59s 46us/step - loss: 0.5515 - accuracy: 0.7076 - val_loss: 0.5678 - val_accuracy: 0.6952
Epoch 4/10
1280000/1280000 [==============================] - 59s 46us/step - loss: 0.5445 - accuracy: 0.7126 - val_loss: 0.5684 - val_accuracy: 0.6955
Epoch 5/10
1280000/1280000 [==============================] - 60s 47us/step - loss: 0.5389 - accuracy: 0.7169 - val_loss: 0.5703 - val_accuracy: 0.6944
Epoch 6/10
1280000/1280000 [==============================] - 60s 47us/step - loss: 0.5338 - accuracy: 0.7204 - val_loss: 0.5733 - val_accuracy: 0.6923
Epoch 7/10
1280000/1280000 [=======

In [50]:
model.save('second_model.h5')

In [53]:
data_string = clean_text("@taylorswift im in love with all your songs!!!!")
data_string = pad_sequences(tokenizer.texts_to_sequences(data_string),
                        maxlen = max_length)

model = load_model('first_model.h5')
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

prediction = model.predict(data_string)

print(prediction)
print(np.argmax(prediction))

[[0.5263099]
 [0.5263099]
 [0.5263099]
 [0.5263099]
 [0.5263099]
 [0.5263099]
 [0.5263099]
 [0.5263099]
 [0.5263099]
 [0.5263099]
 [0.5263099]
 [0.5263099]
 [0.5263099]]
0
